# Atom Mapping Reactions with Kegg Identifiers

To facilitate this, must first match the generated molFiles back to the reactions in the **TECRDB.csv** file. It is important to determine which reactions in which molFiles have been generated for each molecule in the reaction, otherwise there is no point performing atom mapping.

In [42]:
import pandas as pd
import re
import os
import subprocess
from rdkit.Chem import rdChemReactions
from rdkit import Chem
from collections import defaultdict
from rdkit.Chem.rdMolDescriptors import CalcMolFormula
from rdkit.Chem.rdchem import PeriodicTable
from rdkit.Chem.rdchem import GetPeriodicTable
import json

In [46]:
def calculate_total_charge(molecules):
    """Calculate the total charge of a set of molecules."""
    total_charge = 0
    for molecule in molecules:
        for atom in molecule.GetAtoms():
            total_charge += atom.GetFormalCharge()
    return total_charge

def get_reaction_charges(reaction):
    """Return the total charge on the reactant and product side of a reaction."""
    # Parse the reaction

    # Calculate total charge for reactants and products
    reactant_charge = calculate_total_charge(reaction.GetReactants())
    product_charge = calculate_total_charge(reaction.GetProducts())

    return reactant_charge, product_charge

def count_elements(molecule):
    """Count elements in a molecule."""
    element_count = defaultdict(int)
    for atom in molecule.GetAtoms():
        element_count[atom.GetSymbol()] += 1
    return element_count

def is_reaction_balanced_count(file_path):
    """Check if a chemical reaction is balanced."""
    # Parse the reaction
    reaction = rdChemReactions.ReactionFromRxnFile(file_path)

    # Count elements in reactants and products
    reactant_elements = defaultdict(int)
    product_elements = defaultdict(int)

    for reactant in reaction.GetReactants():
        reactant.UpdatePropertyCache(strict=False)
        reactant = Chem.AddHs(reactant)
        for count in count_elements(reactant).items():
            reactant_elements[count[0]] += count[1]

    for product in reaction.GetProducts():
        product.UpdatePropertyCache(strict=False)
        product = Chem.AddHs(product)
        for count in count_elements(product).items():
            product_elements[count[0]] += count[1]
    return reactant_elements == product_elements, (reactant_elements,product_elements)

def is_reaction_balanced_charge(file_path):
    """Check if a chemical reaction is balanced."""
    # Parse the reaction
    reaction = rdChemReactions.ReactionFromRxnFile(file_path)

    # Calculate total charge for reactants and products
    reactant_charge, product_charge = get_reaction_charges(reaction)

    # Compare total charge
    return reactant_charge == product_charge, (reactant_charge, product_charge)

def get_molecular_formula(file_path,direction):
    """Construct the molecular formula of a reaction."""
    # Parse the reaction
    reaction = rdChemReactions.ReactionFromRxnFile(file_path)

    # Initialize a list to store molecular formulas
    reactant_formulas = []
    product_formulas = []
    symb_to_name = {}
    # Loop through reactants and products
    for reactant in reaction.GetReactants():
        try:
            Chem.SanitizeMol(reactant)  # Ensure molecule is properly sanitized
        except:
            pass
        try:
            Chem.AssignStereochemistry(reactant)  # Assign stereochemistry if needed
        except:
            pass
        try:
            reactant.UpdatePropertyCache()
        except:
            pass
        reactant_formulas.append(CalcMolFormula(reactant))
        symb_to_name.update({atom.GetSymbol(): PeriodicTable.GetElementName(GetPeriodicTable(),atom.GetAtomicNum()) for atom in reactant.GetAtoms()})


    for product in reaction.GetProducts():
        try:
            Chem.SanitizeMol(product)  # Ensure molecule is properly sanitized
        except:
            pass
        try:
            Chem.AssignStereochemistry(product)  # Assign stereochemistry if needed
        except:
            pass
        try:
            product.UpdatePropertyCache()
        except:
            pass
        # Chem.GetImplicitHs(product)  # Calculate implicit hydrogens
        product_formulas.append(CalcMolFormula(product))
        symb_to_name.update({atom.GetSymbol(): PeriodicTable.GetElementName(GetPeriodicTable(),atom.GetAtomicNum()) for atom in product.GetAtoms()})

    # Construct reaction formula
    reactant_side = ' + '.join(reactant_formulas)
    product_side = ' + '.join(product_formulas)
    reaction_formula = f"{reactant_side} -> {product_side}" if direction == 'forward' else f"{reactant_side} <=> {reactant_side}"

    return reaction_formula,symb_to_name

def get_reaction_info(rxn_file_path):

    balanced_count,(subs_atoms,prods_atoms) = is_reaction_balanced_count(rxn_file_path)
    balanced_charge,(subs_charge, prods_charge) = is_reaction_balanced_charge(rxn_file_path)
    return balanced_count,(subs_atoms,prods_atoms),balanced_charge, (subs_charge, prods_charge)

def construct_vmh_formula(reaction, subs_abbr, prods_abbr):
    """Construct the VMH formula of a reaction."""
    subs_stch = [float(x) for x in json.loads(reaction.subs_sch)]
    prods_stch = [float(x) for x in json.loads(reaction.prods_sch)]
    subs_comps = json.loads(reaction.subs_comps)
    prods_comps = json.loads(reaction.prods_comps)
    substrate_formula = ' + '.join([f"{subs_stch[i]} {subs_abbr[i]}[{subs_comps[i]}]" for i in range(len(subs_stch))])
    product_formula = ' + '.join([f"{prods_stch[i]} {prods_abbr[i]}[{prods_comps[i]}]" for i in range(len(prods_stch))])
    direction = '->' if reaction.direction == 'forward' else '<=>'
    formula = f"{substrate_formula} {direction} {product_formula}"
    return formula

In [3]:
# import file
data = "/home/jackmcgoldrick/openTECR/data/TECRDB.csv"

In [4]:
# convert to df
openTECR_data = pd.read_csv(data)

In [5]:
openTECR_data

,id,url,reference,method,eval,EC,enzyme_name,reaction,description,K,K_prime,temperature,ionic_strength,p_h,p_mg
0,https://w3id.org/related-to/doi.org/10.5281/ze...,http://xpdb.nist.gov/enzyme_thermodynamics/enz...,07LIN/ALG,spectrophotometry,A,1.1.1.87,homoisocitrate dehydrogenase,kegg:C05662 + kegg:C00003 = kegg:C00322 + kegg...,"(1R,2S)-1-hydroxybutane-1,2,4-tricarboxylate(a...",NaN,0.450,298.15,NaN,7.5,NaN
1,https://w3id.org/related-to/doi.org/10.5281/ze...,http://xpdb.nist.gov/enzyme_thermodynamics/enz...,63GRE,spectrophotometry,C,3.5.4.9,methenyltetrahydrofolate cyclohydrolase,kegg:C00445 + kegg:C00001 = kegg:C00234,"5,10-methenyltetrahydrofolate(aq) + H2O(l) = 1...",NaN,4.200,298.15,NaN,6.5,NaN
2,https://w3id.org/related-to/doi.org/10.5281/ze...,http://xpdb.nist.gov/enzyme_thermodynamics/enz...,67ENG/DEN,spectrophotometry,B,4.2.1.3,aconitate hydratase,kegg:C00311 = kegg:C00158,isocitrate(aq) = citrate(aq),NaN,18.000,310.15,NaN,7.3,2.96
3,https://w3id.org/related-to/doi.org/10.5281/ze...,http://xpdb.nist.gov/enzyme_thermodynamics/enz...,67ENG/DEN,spectrophotometry,B,4.2.1.3,aconitate hydratase,kegg:C00311 = kegg:C00158,isocitrate(aq) = citrate(aq),NaN,25.000,310.15,NaN,7.3,2.80
4,https://w3id.org/related-to/doi.org/10.5281/ze...,http://xpdb.nist.gov/enzyme_thermodynamics/enz...,67ENG/DEN,spectrophotometry,B,4.2.1.3,aconitate hydratase,kegg:C00311 = kegg:C00158,isocitrate(aq) = citrate(aq),NaN,33.000,310.15,NaN,7.3,2.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4539,https://w3id.org/related-to/doi.org/10.5281/ze...,NaN,+92MEY,NaN,E,4.1.1.21,phosphoribosylaminoimidazole carboxylase,kegg:C03373 + kegg:C00288 = kegg:C04751 + kegg...,Aminoimidazole ribotide + CO2 = 1-(5-Phospho-D...,NaN,1.800,310.15,NaN,7.8,NaN
4540,https://w3id.org/related-to/doi.org/10.5281/ze...,NaN,+00WAL,NaN,E,2.1.2.3,phosphoribosylaminoimidazolecarboxamide formyl...,kegg:C00234 + kegg:C04677 = kegg:C00101 + kegg...,10-Formyltetrahydrofolate + 1-(5'-Phosphoribos...,NaN,0.024,298.15,NaN,7.5,NaN
4541,https://w3id.org/related-to/doi.org/10.5281/ze...,NaN,+59BLA,NaN,E,4.3.-.-,formaldehyde condensation with THF,kegg:C00101 + kegg:C00067 = kegg:C00143,"THF(aq) + formaldehyde(aq) = 5,10-CH2-THF(aq)",NaN,7700.000,293.15,NaN,7.2,NaN
4542,https://w3id.org/related-to/doi.org/10.5281/ze...,NaN,+74FER,NaN,E,4.1.2.43,3-Hexulose-6-Phosphate synthase,kegg:C00199 + kegg:C00067 = kegg:C06019,D-Ribulose 5-phosphate + Formaldehyde = D-arab...,NaN,25000.000,303.15,NaN,7.0,NaN


In [5]:
# extract 'reaction' column
kegg_reactions = openTECR_data['reaction']

In [6]:
kegg_reactions

0       kegg:C05662 + kegg:C00003 = kegg:C00322 + kegg...
1                 kegg:C00445 + kegg:C00001 = kegg:C00234
2                               kegg:C00311 = kegg:C00158
3                               kegg:C00311 = kegg:C00158
4                               kegg:C00311 = kegg:C00158
                              ...                        
4539    kegg:C03373 + kegg:C00288 = kegg:C04751 + kegg...
4540    kegg:C00234 + kegg:C04677 = kegg:C00101 + kegg...
4541              kegg:C00101 + kegg:C00067 = kegg:C00143
4542              kegg:C00199 + kegg:C00067 = kegg:C06019
4543                            kegg:C06019 = kegg:C00085
Name: reaction, Length: 4544, dtype: object

In [18]:
type(kegg_reactions)

pandas.core.series.Series

In [9]:
## path to molFiles
molFiles = "/home/jackmcgoldrick/openTECR/data/molFiles_Final"

In [10]:
list_ofMols = []
# Loop through all files in the directory
for filename in os.listdir(molFiles):
    filepath = os.path.join(molFiles, filename)
    if os.path.isfile(filepath):  # Check if it's a file
        print(f"Processing file: {filename}")
        list_ofMols.append(filename)

Processing file: C00294.mol
Processing file: C02045.mol
Processing file: C00333.mol
Processing file: C00168.mol
Processing file: C03360.mol
Processing file: C00810.mol
Processing file: C00441.mol
Processing file: C00259.mol
Processing file: C01931.mol
Processing file: C08126.mol
Processing file: C00007.mol
Processing file: C00222.mol
Processing file: C00322.mol
Processing file: C00191.mol
Processing file: C00170.mol
Processing file: C06468.mol
Processing file: C00877.mol
Processing file: C00474.mol
Processing file: C00379.mol
Processing file: C03521.mol
Processing file: C00074.mol
Processing file: C03410.mol
Processing file: C01107.mol
Processing file: C00334.mol
Processing file: C01005.mol
Processing file: C02048.mol
Processing file: C02557.mol
Processing file: C01019.mol
Processing file: C00159.mol
Processing file: C03351.mol
Processing file: C00231.mol
Processing file: C03314.mol
Processing file: C01231.mol
Processing file: C00158.mol
Processing file: C02480.mol
Processing file: C00

In [11]:
list_ofMols

['C00294.mol',
 'C02045.mol',
 'C00333.mol',
 'C00168.mol',
 'C03360.mol',
 'C00810.mol',
 'C00441.mol',
 'C00259.mol',
 'C01931.mol',
 'C08126.mol',
 'C00007.mol',
 'C00222.mol',
 'C00322.mol',
 'C00191.mol',
 'C00170.mol',
 'C06468.mol',
 'C00877.mol',
 'C00474.mol',
 'C00379.mol',
 'C03521.mol',
 'C00074.mol',
 'C03410.mol',
 'C01107.mol',
 'C00334.mol',
 'C01005.mol',
 'C02048.mol',
 'C02557.mol',
 'C01019.mol',
 'C00159.mol',
 'C03351.mol',
 'C00231.mol',
 'C03314.mol',
 'C01231.mol',
 'C00158.mol',
 'C02480.mol',
 'C00100.mol',
 'C00463.mol',
 'C00236.mol',
 'C00349.mol',
 'C00507.mol',
 'C16512.mol',
 'C00414.mol',
 'C01213.mol',
 'C16310.mol',
 'C05437.mol',
 'C01650.mol',
 'C02666.mol',
 'C01083.mol',
 'C00922.mol',
 'C00227.mol',
 'C01595.mol',
 'C00475.mol',
 'C00146.mol',
 'C03448.mol',
 'C04114.mol',
 'C00363.mol',
 'C00756.mol',
 'C01172.mol',
 'C05979.mol',
 'C00449.mol',
 'C00870.mol',
 'C00940.mol',
 'C00026.mol',
 'C06322.mol',
 'C04618.mol',
 'C00133.mol',
 'C00580.m

In [12]:
# Regex pattern to remove `.mol` extension
pattern = r"\.mol$"

# Apply regex to each filename
keggIDs = [re.sub(pattern, "", filename) for filename in list_ofMols]


In [13]:
keggIDs

['C00294',
 'C02045',
 'C00333',
 'C00168',
 'C03360',
 'C00810',
 'C00441',
 'C00259',
 'C01931',
 'C08126',
 'C00007',
 'C00222',
 'C00322',
 'C00191',
 'C00170',
 'C06468',
 'C00877',
 'C00474',
 'C00379',
 'C03521',
 'C00074',
 'C03410',
 'C01107',
 'C00334',
 'C01005',
 'C02048',
 'C02557',
 'C01019',
 'C00159',
 'C03351',
 'C00231',
 'C03314',
 'C01231',
 'C00158',
 'C02480',
 'C00100',
 'C00463',
 'C00236',
 'C00349',
 'C00507',
 'C16512',
 'C00414',
 'C01213',
 'C16310',
 'C05437',
 'C01650',
 'C02666',
 'C01083',
 'C00922',
 'C00227',
 'C01595',
 'C00475',
 'C00146',
 'C03448',
 'C04114',
 'C00363',
 'C00756',
 'C01172',
 'C05979',
 'C00449',
 'C00870',
 'C00940',
 'C00026',
 'C06322',
 'C04618',
 'C00133',
 'C00580',
 'C05385',
 'C00617',
 'C00376',
 'C00025',
 'C01644',
 'C00920',
 'C03939',
 'C00232',
 'C00016',
 'C00256',
 'C00181',
 'C03875',
 'C01721',
 'C00588',
 'C03493',
 'C00327',
 'C00361',
 'C00224',
 'C02780',
 'C00250',
 'C06219',
 'C01175',
 'C00117',
 'C00004',

In [14]:
# Function to extract KEGG identifiers (ignoring coefficients)
def extract_kegg_ids(reaction):
    return re.findall(r"kegg:(C\d+)", reaction)

In [15]:
# Function to check if all molecules in a reaction have mol files
def has_all_molfiles(reaction):
    molecules = extract_kegg_ids(reaction)  # Extract molecule IDs
    return all(mol in keggIDs for mol in molecules)

# Apply the filter to the DataFrame
rxns = openTECR_data[openTECR_data["reaction"].apply(has_all_molfiles)]

# Display the filtered DataFrame
print(rxns)

                                                     id  \
0     https://w3id.org/related-to/doi.org/10.5281/ze...   
2     https://w3id.org/related-to/doi.org/10.5281/ze...   
3     https://w3id.org/related-to/doi.org/10.5281/ze...   
4     https://w3id.org/related-to/doi.org/10.5281/ze...   
5     https://w3id.org/related-to/doi.org/10.5281/ze...   
...                                                 ...   
4538  https://w3id.org/related-to/doi.org/10.5281/ze...   
4540  https://w3id.org/related-to/doi.org/10.5281/ze...   
4541  https://w3id.org/related-to/doi.org/10.5281/ze...   
4542  https://w3id.org/related-to/doi.org/10.5281/ze...   
4543  https://w3id.org/related-to/doi.org/10.5281/ze...   

                                                    url   reference  \
0     http://xpdb.nist.gov/enzyme_thermodynamics/enz...   07LIN/ALG   
2     http://xpdb.nist.gov/enzyme_thermodynamics/enz...   67ENG/DEN   
3     http://xpdb.nist.gov/enzyme_thermodynamics/enz...   67ENG/DEN   
4     h

In [16]:
type(rxns)

pandas.core.frame.DataFrame

In [17]:
rxns.to_csv("/home/jackmcgoldrick/openTECR/results/rxns_withAssoc_molFiles.csv")

## Cleaning Reaction Names for Entry into Cobra Model

In [10]:
# Remove the 'kegg:' prefix from the entire column
openTECR_data['reaction'] = openTECR_data['reaction'].str.replace('kegg:', '', regex=False)

print(openTECR_data)


                                                     id  \
0     https://w3id.org/related-to/doi.org/10.5281/ze...   
1     https://w3id.org/related-to/doi.org/10.5281/ze...   
2     https://w3id.org/related-to/doi.org/10.5281/ze...   
3     https://w3id.org/related-to/doi.org/10.5281/ze...   
4     https://w3id.org/related-to/doi.org/10.5281/ze...   
...                                                 ...   
4539  https://w3id.org/related-to/doi.org/10.5281/ze...   
4540  https://w3id.org/related-to/doi.org/10.5281/ze...   
4541  https://w3id.org/related-to/doi.org/10.5281/ze...   
4542  https://w3id.org/related-to/doi.org/10.5281/ze...   
4543  https://w3id.org/related-to/doi.org/10.5281/ze...   

                                                    url  reference  \
0     http://xpdb.nist.gov/enzyme_thermodynamics/enz...  07LIN_ALG   
1     http://xpdb.nist.gov/enzyme_thermodynamics/enz...      63GRE   
2     http://xpdb.nist.gov/enzyme_thermodynamics/enz...  67ENG_DEN   
3     http:

In [6]:
openTECR_data['reference'] = openTECR_data['reference'].str.replace('/', '_', regex=False)


In [8]:
openTECR_data['reference'] = openTECR_data['reference'].str.replace('+', '', regex=False)


In [11]:
openTECR_data

,id,url,reference,method,eval,EC,enzyme_name,reaction,description,K,K_prime,temperature,ionic_strength,p_h,p_mg
0,https://w3id.org/related-to/doi.org/10.5281/ze...,http://xpdb.nist.gov/enzyme_thermodynamics/enz...,07LIN_ALG,spectrophotometry,A,1.1.1.87,homoisocitrate dehydrogenase,C05662 + C00003 = C00322 + C00288 + C00004,"(1R,2S)-1-hydroxybutane-1,2,4-tricarboxylate(a...",NaN,0.450,298.15,NaN,7.5,NaN
1,https://w3id.org/related-to/doi.org/10.5281/ze...,http://xpdb.nist.gov/enzyme_thermodynamics/enz...,63GRE,spectrophotometry,C,3.5.4.9,methenyltetrahydrofolate cyclohydrolase,C00445 + C00001 = C00234,"5,10-methenyltetrahydrofolate(aq) + H2O(l) = 1...",NaN,4.200,298.15,NaN,6.5,NaN
2,https://w3id.org/related-to/doi.org/10.5281/ze...,http://xpdb.nist.gov/enzyme_thermodynamics/enz...,67ENG_DEN,spectrophotometry,B,4.2.1.3,aconitate hydratase,C00311 = C00158,isocitrate(aq) = citrate(aq),NaN,18.000,310.15,NaN,7.3,2.96
3,https://w3id.org/related-to/doi.org/10.5281/ze...,http://xpdb.nist.gov/enzyme_thermodynamics/enz...,67ENG_DEN,spectrophotometry,B,4.2.1.3,aconitate hydratase,C00311 = C00158,isocitrate(aq) = citrate(aq),NaN,25.000,310.15,NaN,7.3,2.80
4,https://w3id.org/related-to/doi.org/10.5281/ze...,http://xpdb.nist.gov/enzyme_thermodynamics/enz...,67ENG_DEN,spectrophotometry,B,4.2.1.3,aconitate hydratase,C00311 = C00158,isocitrate(aq) = citrate(aq),NaN,33.000,310.15,NaN,7.3,2.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4539,https://w3id.org/related-to/doi.org/10.5281/ze...,NaN,92MEY,NaN,E,4.1.1.21,phosphoribosylaminoimidazole carboxylase,C03373 + C00288 = C04751 + C00001,Aminoimidazole ribotide + CO2 = 1-(5-Phospho-D...,NaN,1.800,310.15,NaN,7.8,NaN
4540,https://w3id.org/related-to/doi.org/10.5281/ze...,NaN,00WAL,NaN,E,2.1.2.3,phosphoribosylaminoimidazolecarboxamide formyl...,C00234 + C04677 = C00101 + C04734,10-Formyltetrahydrofolate + 1-(5'-Phosphoribos...,NaN,0.024,298.15,NaN,7.5,NaN
4541,https://w3id.org/related-to/doi.org/10.5281/ze...,NaN,59BLA,NaN,E,4.3.-.-,formaldehyde condensation with THF,C00101 + C00067 = C00143,"THF(aq) + formaldehyde(aq) = 5,10-CH2-THF(aq)",NaN,7700.000,293.15,NaN,7.2,NaN
4542,https://w3id.org/related-to/doi.org/10.5281/ze...,NaN,74FER,NaN,E,4.1.2.43,3-Hexulose-6-Phosphate synthase,C00199 + C00067 = C06019,D-Ribulose 5-phosphate + Formaldehyde = D-arab...,NaN,25000.000,303.15,NaN,7.0,NaN


In [12]:
openTECR_data.to_csv('/home/jackmcgoldrick/openTECR/results/openTECR_kegggRxns_cleaned.csv')

In [13]:
# further clean reactions to replace = with <=>
openTECR_data['reaction'] = openTECR_data['reaction'].str.replace('=', '<=>', regex=False)

print(openTECR_data)

                                                     id  \
0     https://w3id.org/related-to/doi.org/10.5281/ze...   
1     https://w3id.org/related-to/doi.org/10.5281/ze...   
2     https://w3id.org/related-to/doi.org/10.5281/ze...   
3     https://w3id.org/related-to/doi.org/10.5281/ze...   
4     https://w3id.org/related-to/doi.org/10.5281/ze...   
...                                                 ...   
4539  https://w3id.org/related-to/doi.org/10.5281/ze...   
4540  https://w3id.org/related-to/doi.org/10.5281/ze...   
4541  https://w3id.org/related-to/doi.org/10.5281/ze...   
4542  https://w3id.org/related-to/doi.org/10.5281/ze...   
4543  https://w3id.org/related-to/doi.org/10.5281/ze...   

                                                    url  reference  \
0     http://xpdb.nist.gov/enzyme_thermodynamics/enz...  07LIN_ALG   
1     http://xpdb.nist.gov/enzyme_thermodynamics/enz...      63GRE   
2     http://xpdb.nist.gov/enzyme_thermodynamics/enz...  67ENG_DEN   
3     http:

In [14]:
openTECR_data.to_csv('/home/jackmcgoldrick/openTECR/results/openTECR_kegggRxns_cleaned.csv')

want to remove duplicate reactions, then use reference assoc with rxn to name the mapped molecule

for each file currently in /home/jackmcgoldrick/openTECR/results/AM_mapping/unMapped want to run RDT on them

In [37]:
# path to save the files to 
results_AM = '/home/jackmcgoldrick/openTECR/results/Atom_Mapping_py'

In [35]:
import os
import subprocess
import shutil

def RDT(rxn_file_path, output_dir='.'):  
    """
    Processes a chemical reaction using RDT to generate atom-atom mapping and visualization.
    
    Input:
    - rxn_file_path (str): Path to the input RXN file.
    - output_dir (str): Directory where output files will be saved (default: current directory).

    Output:
    - (dict): Dictionary containing the path to the reaction visualization image and a flag indicating if it's found in VMH.
    """
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Extract the base name of the input file without extension
    base_name = os.path.splitext(os.path.basename(rxn_file_path))[0]

    # Define output file names based on input file name
    destination_path_png = os.path.join(output_dir, f"{base_name}.png")
    destination_path_rxn = os.path.join(output_dir, f"{base_name}.rxn")

    # Locate the RDT jar file (adjust the path as needed)
    base_dir = os.getcwd()  # Use current working directory as a fallback
    rdt_path = '/home/jackmcgoldrick/Downloads/rdt-2.4.1-jar-with-dependencies.jar'

    # Command to execute the RDT jar
    cmd = ['java', '-jar', rdt_path, '-Q', 'RXN', '-q', rxn_file_path, '-g', '-j', 'AAM', '-f', 'XML']

    # Use the output directory as the working directory
    subprocess.Popen(cmd, cwd=output_dir).wait()

    # Temporary output file names generated by RDT
    temp_img_path = os.path.join(output_dir, 'ECBLAST_temp_AAM.png')
    temp_rxn_file_path = os.path.join(output_dir, 'ECBLAST_temp_AAM.rxn')

    # Move the temporary files to the specified destination paths
    shutil.move(temp_img_path, destination_path_png)
    shutil.move(temp_rxn_file_path, destination_path_rxn)

    # Return the constructed paths
    return {
        "visualizations": [destination_path_png],
        "vmh_found": False
    }


In [36]:
rxn_files = '/home/jackmcgoldrick/openTECR/results/AM_mapping/unMapped'

In [53]:
import os

for file in os.listdir(rxn_files):
    full_path = os.path.join(rxn_files, file)  # Construct the full file path
    if os.path.isfile(full_path):  # Check if it's a file (and not a directory)
        try:
            atomMapping = RDT(full_path, results_AM)  # Pass the full file path
            print(f"Processed {file}: {atomMapping}")
        except Exception as e:
            print(f"Error processing {file}: {e}")
    else:
        print(f"Skipping non-file item: {file}")


!--------------------------------------------------------
Reaction Decoder Tool (RDT)

Author: Syed Asad Rahman
e-mail: asad@ebi.ac.uk|s9asad@gmail.com
c/o EMBL-European BioInformatics Institute (EBI)
WTGC, CB10 1SD Hinxton
UK

Note: The copyright of this software belongs to the author
and EMBL-European BioInformatics Institute (EBI).

Reference
Rahman, S.A. et.al.(2016) Reaction Decoder Tool (RDT):
Extracting Features from Chemical Reactions, Bioinformatics (2016)
doi: 10.1093/bioinformatics/btw096
!--------------------------------------------------------

-- AAM --
0    [main] INFO  net.sf.jnati.deploy.artefact.ConfigManager  - Loading global configuration
4    [main] DEBUG net.sf.jnati.deploy.artefact.ConfigManager  - Loading defaults: jar:file:/home/jackmcgoldrick/Downloads/rdt-2.4.1-jar-with-dependencies.jar!/META-INF/jnati/jnati.default-properties
5    [main] INFO  net.sf.jnati.deploy.artefact.ConfigManager  - Loading artefact configuration: jniinchi-1.03_1
5    [main] DEBUG net.

uk.ac.ebi.aamtool.ReactionDecoder ERROR: SEVERE CommandLine option parsing error: java.lang.ArrayIndexOutOfBoundsException: Index 0 out of bounds for length 0
uk.ac.ebi.aamtool.ReactionDecoder ERROR: SEVERE CommandLine option parsing error: java.lang.ArrayIndexOutOfBoundsException: Index 0 out of bounds for length 0


Error processing 89ROM_DEM.rxn: [Errno 2] No such file or directory: '/home/jackmcgoldrick/openTECR/results/Atom_Mapping_py/ECBLAST_temp_AAM.png'
!--------------------------------------------------------
Reaction Decoder Tool (RDT)

Author: Syed Asad Rahman
e-mail: asad@ebi.ac.uk|s9asad@gmail.com
c/o EMBL-European BioInformatics Institute (EBI)
WTGC, CB10 1SD Hinxton
UK

Note: The copyright of this software belongs to the author
and EMBL-European BioInformatics Institute (EBI).

Reference
Rahman, S.A. et.al.(2016) Reaction Decoder Tool (RDT):
Extracting Features from Chemical Reactions, Bioinformatics (2016)
doi: 10.1093/bioinformatics/btw096
!--------------------------------------------------------

-- AAM --
0    [main] INFO  net.sf.jnati.deploy.artefact.ConfigManager  - Loading global configuration
3    [main] DEBUG net.sf.jnati.deploy.artefact.ConfigManager  - Loading defaults: jar:file:/home/jackmcgoldrick/Downloads/rdt-2.4.1-jar-with-dependencies.jar!/META-INF/jnati/jnati.default

uk.ac.ebi.aamtool.ReactionDecoder ERROR: SEVERE CommandLine option parsing error: java.lang.ArrayIndexOutOfBoundsException: Index 0 out of bounds for length 0


0    [main] INFO  net.sf.jnati.deploy.artefact.ConfigManager  - Loading global configuration
2    [main] DEBUG net.sf.jnati.deploy.artefact.ConfigManager  - Loading defaults: jar:file:/home/jackmcgoldrick/Downloads/rdt-2.4.1-jar-with-dependencies.jar!/META-INF/jnati/jnati.default-properties
3    [main] INFO  net.sf.jnati.deploy.artefact.ConfigManager  - Loading artefact configuration: jniinchi-1.03_1
3    [main] DEBUG net.sf.jnati.deploy.artefact.ConfigManager  - Loading instance defaults: jar:file:/home/jackmcgoldrick/Downloads/rdt-2.4.1-jar-with-dependencies.jar!/META-INF/jnati/jnati.instance.default-properties
4    [main] INFO  net.sf.jnati.deploy.repository.ClasspathRepository  - Searching classpath for: jniinchi-1.03_1-LINUX-AMD64
5    [main] INFO  net.sf.jnati.deploy.repository.LocalRepository  - Searching local repository for: jniinchi-1.03_1-LINUX-AMD64
5    [main] DEBUG net.sf.jnati.deploy.repository.LocalRepository  - Artefact path: /home/jackmcgoldrick/.jnati/repo/jniinchi/1

...

0    [main] INFO  net.sf.jnati.deploy.artefact.ConfigManager  - Loading global configuration
2    [main] DEBUG net.sf.jnati.deploy.artefact.ConfigManager  - Loading defaults: jar:file:/home/jackmcgoldrick/Downloads/rdt-2.4.1-jar-with-dependencies.jar!/META-INF/jnati/jnati.default-properties
2    [main] INFO  net.sf.jnati.deploy.artefact.ConfigManager  - Loading artefact configuration: jniinchi-1.03_1
3    [main] DEBUG net.sf.jnati.deploy.artefact.ConfigManager  - Loading instance defaults: jar:file:/home/jackmcgoldrick/Downloads/rdt-2.4.1-jar-with-dependencies.jar!/META-INF/jnati/jnati.instance.default-properties
3    [main] INFO  net.sf.jnati.deploy.repository.ClasspathRepository  - Searching classpath for: jniinchi-1.03_1-LINUX-AMD64
4    [main] INFO  net.sf.jnati.deploy.repository.LocalRepository  - Searching local repository for: jniinchi-1.03_1-LINUX-AMD64
4    [main] DEBUG net.sf.jnati.deploy.repository.LocalRepository  - Artefact path: /home/jackmcgoldrick/.jnati/repo/jniinchi/1

uk.ac.ebi.reactionblast.mechanism.ReactionMechanismTool WARN: Unmapped atoms present in this reaction(MIN) algorithm.


0    [main] INFO  net.sf.jnati.deploy.artefact.ConfigManager  - Loading global configuration
2    [main] DEBUG net.sf.jnati.deploy.artefact.ConfigManager  - Loading defaults: jar:file:/home/jackmcgoldrick/Downloads/rdt-2.4.1-jar-with-dependencies.jar!/META-INF/jnati/jnati.default-properties
3    [main] INFO  net.sf.jnati.deploy.artefact.ConfigManager  - Loading artefact configuration: jniinchi-1.03_1
3    [main] DEBUG net.sf.jnati.deploy.artefact.ConfigManager  - Loading instance defaults: jar:file:/home/jackmcgoldrick/Downloads/rdt-2.4.1-jar-with-dependencies.jar!/META-INF/jnati/jnati.instance.default-properties
4    [main] INFO  net.sf.jnati.deploy.repository.ClasspathRepository  - Searching classpath for: jniinchi-1.03_1-LINUX-AMD64
5    [main] INFO  net.sf.jnati.deploy.repository.LocalRepository  - Searching local repository for: jniinchi-1.03_1-LINUX-AMD64
5    [main] DEBUG net.sf.jnati.deploy.repository.LocalRepository  - Artefact path: /home/jackmcgoldrick/.jnati/repo/jniinchi/1

uk.ac.ebi.aamtool.ReactionDecoder ERROR: SEVERE CommandLine option parsing error: java.lang.ArrayIndexOutOfBoundsException: Index 0 out of bounds for length 0


0    [main] INFO  net.sf.jnati.deploy.artefact.ConfigManager  - Loading global configuration
3    [main] DEBUG net.sf.jnati.deploy.artefact.ConfigManager  - Loading defaults: jar:file:/home/jackmcgoldrick/Downloads/rdt-2.4.1-jar-with-dependencies.jar!/META-INF/jnati/jnati.default-properties
3    [main] INFO  net.sf.jnati.deploy.artefact.ConfigManager  - Loading artefact configuration: jniinchi-1.03_1
4    [main] DEBUG net.sf.jnati.deploy.artefact.ConfigManager  - Loading instance defaults: jar:file:/home/jackmcgoldrick/Downloads/rdt-2.4.1-jar-with-dependencies.jar!/META-INF/jnati/jnati.instance.default-properties
4    [main] INFO  net.sf.jnati.deploy.repository.ClasspathRepository  - Searching classpath for: jniinchi-1.03_1-LINUX-AMD64
5    [main] INFO  net.sf.jnati.deploy.repository.LocalRepository  - Searching local repository for: jniinchi-1.03_1-LINUX-AMD64
5    [main] DEBUG net.sf.jnati.deploy.repository.LocalRepository  - Artefact path: /home/jackmcgoldrick/.jnati/repo/jniinchi/1

In [54]:
import os
import shutil

def organize_files_by_extension(directory):
    """
    Organizes files in a directory based on their extensions.
    - .xml files are moved to the 'xml' folder.
    - .png files are moved to the 'images' folder.
    - .rxn files are moved to the 'AtomMapped' folder.
    
    Input:
    - directory (str): The path to the directory containing the files to organize.
    """
    # Define subdirectories for different file types
    subdirectories = {
        '.xml': 'xml',
        '.png': 'images',
        '.rxn': 'AtomMapped',
    }

    # Ensure subdirectories exist
    for subdir in subdirectories.values():
        subdir_path = os.path.join(directory, subdir)
        if not os.path.exists(subdir_path):
            os.makedirs(subdir_path)

    # Iterate through files in the directory
    for file in os.listdir(directory):
        file_path = os.path.join(directory, file)

        # Skip directories
        if os.path.isdir(file_path):
            continue

        # Get the file extension
        _, extension = os.path.splitext(file)

        # Check if the extension matches one of the defined subdirectories
        if extension in subdirectories:
            dest_dir = os.path.join(directory, subdirectories[extension])
            dest_path = os.path.join(dest_dir, file)

            # Move the file
            shutil.move(file_path, dest_path)
            print(f"Moved {file} to {dest_dir}")




In [55]:
# organising the files into diff subfolders based on extension

path_to = "/home/jackmcgoldrick/openTECR/results/Atom_Mapping_py"

organize_files_by_extension(path_to)

Moved ECBLAST_56RAB_PRI_AAM.rxn to /home/jackmcgoldrick/openTECR/results/Atom_Mapping_py/AtomMapped
Moved ECBLAST_65SHA_CLE_AAM.xml to /home/jackmcgoldrick/openTECR/results/Atom_Mapping_py/xml
Moved ECBLAST_83KAT_SUE_AAM.rxn to /home/jackmcgoldrick/openTECR/results/Atom_Mapping_py/AtomMapped
Moved ECBLAST_62RAV_WOL2_AAM.rxn to /home/jackmcgoldrick/openTECR/results/Atom_Mapping_py/AtomMapped
Moved ECBLAST_74FER_STR_AAM.xml to /home/jackmcgoldrick/openTECR/results/Atom_Mapping_py/xml
Moved ECBLAST_40HER_GOR_AAM.xml to /home/jackmcgoldrick/openTECR/results/Atom_Mapping_py/xml
Moved ECBLAST_60PRI_HOR_AAM.png to /home/jackmcgoldrick/openTECR/results/Atom_Mapping_py/images
Moved ECBLAST_93VIN_GRU_AAM.xml to /home/jackmcgoldrick/openTECR/results/Atom_Mapping_py/xml
Moved ECBLAST_84TEW_GOL_AAM.rxn to /home/jackmcgoldrick/openTECR/results/Atom_Mapping_py/AtomMapped
Moved ECBLAST_37ADL_SRE_AAM.rxn to /home/jackmcgoldrick/openTECR/results/Atom_Mapping_py/AtomMapped
Moved ECBLAST_75MUR_TSU_AAM.png

In [56]:
import os
import shutil
# dir to rxn files 
rxns = '/home/jackmcgoldrick/openTECR/results/Atom_Mapping_py/AtomMapped'

# Directory to move unbalanced files
unbalanced_dir = '/home/jackmcgoldrick/openTECR/results/Atom_Mapping_py/unbalanced'

# Ensure the unbalanced directory exists
if not os.path.exists(unbalanced_dir):
    os.makedirs(unbalanced_dir)

# Process each file in the directory
for file in os.listdir(rxns):
    full_path = os.path.join(rxns, file)  # Construct the full file path

    if os.path.isfile(full_path):  # Check if it's a file (and not a directory)
        try:
            # Check if the reaction is balanced
            check_balance = get_reaction_info(full_path)
            if not check_balance:  # If not balanced, move the file
                dest_path = os.path.join(unbalanced_dir, file)
                shutil.move(full_path, dest_path)
                print(f"Moved unbalanced file {file} to {unbalanced_dir}")
            else:
                print(f"File {file} is balanced.")
        except Exception as e:
            print(f"Error processing {file}: {e}")
    else:
        print(f"Skipping non-file item: {file}")


File ECBLAST_56RAB_PRI_AAM.rxn is balanced.
File ECBLAST_83KAT_SUE_AAM.rxn is balanced.
File ECBLAST_62RAV_WOL2_AAM.rxn is balanced.
File ECBLAST_84TEW_GOL_AAM.rxn is balanced.
File ECBLAST_37ADL_SRE_AAM.rxn is balanced.
File ECBLAST_62HIM_RAB_AAM.rxn is balanced.
File ECBLAST_76HIL_ATT_AAM.rxn is balanced.
File ECBLAST_72COO_MEI_AAM.rxn is balanced.
File ECBLAST_64BAR_ROO_AAM.rxn is balanced.
File ECBLAST_64ROS_RAP_AAM.rxn is balanced.
File ECBLAST_52LEL_CAR_AAM.rxn is balanced.
File ECBLAST_59HOL_AAM.rxn is balanced.
File ECBLAST_56SMI_STA_AAM.rxn is balanced.
File ECBLAST_73HAV_PIT_AAM.rxn is balanced.
File ECBLAST_62UEH_AAM.rxn is balanced.
File ECBLAST_61BER_BER_AAM.rxn is balanced.
File ECBLAST_66BER_MOE_AAM.rxn is balanced.
File ECBLAST_65KAZ_GRO_AAM.rxn is balanced.
File ECBLAST_70BRO_KRE_AAM.rxn is balanced.
File ECBLAST_86GRA_ELL_AAM.rxn is balanced.
File ECBLAST_85TEW_STE_AAM.rxn is balanced.
File ECBLAST_73SUG_AAM.rxn is balanced.
File ECBLAST_66MAR_WAD_AAM.rxn is balanced.